In [ ]:
# THIS CODE WAS PARTIALLY CREATED USING CHATGPT

In [1]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 11868    0 11868    0     0  57776      0 --:--:-- --:--:-- --:--:-- 57892
######################################################################## 100.0%#=#=#                                                                          #######################                80.6%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!pip install ollama

In [3]:
import subprocess

In [58]:
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
process2 = subprocess.Popen(["ollama", "run", "llava:13b-v1.6-vicuna-q2_K"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

In [33]:
!ollama run llava:13b-v1.6-vicuna-q2_K

Error: could not connect to ollama app, is it running?


In [59]:
!ollama list

NAME                      	ID          	SIZE  	MODIFIED       
llava:13b-v1.6-vicuna-q2_K	87aecb135e2a	5.5 GB	34 minutes ago	
llava:34b-v1.6-q2_K       	6326f59da4f1	13 GB 	50 minutes ago	


In [60]:
from PIL import Image

In [61]:
!git clone https://github.com/sahal-mulki/geo-llm.git

fatal: destination path 'geo-llm' already exists and is not an empty directory.


In [62]:
import os
import csv
from PIL import Image
from io import BytesIO
from ollama import generate, chat
from tqdm import tqdm

# Function to read the image from a file
def read_image_file(file_path):
    with open(file_path, 'rb') as file:
        return file.read()

# Function to explain the image using the llava model
def explain_image(image_data, addresses):
    prompt = (
        "You are an Image OSINT Investigator. I will provide you with an image and it's your job to determine where it was taken. "
        "By the end of your investigation you have to have at least a rough guess of the image's location. "
        "DECIDE ON ONE FINAL LOCATION, EVEN IF IT'S WRONG, JUST GUESS. "
        "Here's some AI-generated addresses that could be wrong to get you started. Choose one of these if you're uncertain:\n"
        f"1. {addresses[0]}\n"
        f"2. {addresses[1]}\n"
        f"3. {addresses[2]}\n"
        f"4. {addresses[3]}\n"
        f"5. {addresses[4]}"
    )
    print(prompt)
    explanation = ""
    for response in generate('llava:13b-v1.6-vicuna-q2_K', prompt, images=[image_data], stream=True):
        explanation += response['response']
    print(explanation)
    return explanation

# Function to chat with the model based on an initial message
def chat_with_model(initial_message):
    messages = [
        {
            'role': 'user',
            'content': initial_message,
        },
    ]

    response = chat('llava:13b-v1.6-vicuna-q2_K', messages=messages)
    return response['message']['content']

# Function to process a single image and return the results
def process_image(image_file_path, additional_info):
    image_data = read_image_file(image_file_path)
    
    list_of_addresses = [additional_info['Prediction 1 Address'], additional_info['Prediction 2 Address'], additional_info['Prediction 3 Address'],additional_info['Prediction 4 Address'], additional_info['Prediction 5 Address']]
    
    print(list_of_addresses)
    
    explanation = explain_image(image_data, list_of_addresses)
    
    final_prompt = explanation + "\nSummarize the above in JSON as such: '{\"image_loc\": \"______, _____, country\"}'"
    chat_response = chat_with_model(final_prompt)
    
    return explanation, chat_response

# Main function to iterate over a folder of images and save results to a CSV file
def process_folder(image_folder_path, output_csv_path, reference_csv_path):
    # Load the reference CSV file into a DataFrame
    reference_df = pd.read_csv(reference_csv_path)

    # Open the output CSV file in append mode
    with open(output_csv_path, mode='a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["image_file", "explanation", "chat_response"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Check if the CSV file is empty and write the header if it is
        if os.stat(output_csv_path).st_size == 0:
            writer.writeheader()

        for image_file in tqdm(os.listdir(image_folder_path)):
            image_file_path = os.path.join(image_folder_path, image_file)
            if os.path.isfile(image_file_path):
                # Find the corresponding record in the reference CSV
                reference_record = reference_df[reference_df['Image'] == image_file]
                if not reference_record.empty:
                    additional_info = reference_record.iloc[0].to_dict()
                    explanation, chat_response = process_image(image_file_path, additional_info)
                    writer.writerow({
                        "image_file": image_file,
                        "explanation": explanation,
                        "chat_response": chat_response,
                    })
                    print(chat_response)
                else:
                    print(f"No reference record found for image {image_file}")


In [63]:
import pandas as pd

In [ ]:
process_folder('/kaggle/input/llms-for-image-geolocation-benchmarks/im2gps3k_places365/im2gps3ktest', '/kaggle/working/imp2gps3k_output.csv', '/kaggle/working/geo-llm/geoclip_predictions_im2gps3k.csv')

  0%|          | 0/3000 [00:00<?, ?it/s]

['Port Macquarie-Hastings Council, New South Wales, Australia', 'Singleton Council, New South Wales, 2330, Australia', 'Noosa Shire, Queensland, Australia', 'Mareeba Shire, Queensland, Australia', 'Gold Coast City, Queensland, Australia']
You are an Image OSINT Investigator. I will provide you with an image and it's your job to determine where it was taken. By the end of your investigation you have to have at least a rough guess of the image's location. DECIDE ON ONE FINAL LOCATION, EVEN IF IT'S WRONG, JUST GUESS. Here's some AI-generated addresses that could be wrong to get you started. Choose one of these if you're uncertain:
1. Port Macquarie-Hastings Council, New South Wales, Australia
2. Singleton Council, New South Wales, 2330, Australia
3. Noosa Shire, Queensland, Australia
4. Mareeba Shire, Queensland, Australia
5. Gold Coast City, Queensland, Australia
 Given the context, I would guess that the image was taken in Australia, likely in an area associated with kangaroo or wildlif

  0%|          | 1/3000 [00:17<14:41:29, 17.64s/it]

 ```json
{
  "image_loc": "Port Macquarie-Hastings Council, New South Wales, Australia"
}
```
['Okaloosa County, Florida, United States', 'Gurugram, Haryana, India', 'Presidente Hayes, Región Occidental, Paraguay', 'Farrukhnagar, Gurugram, Haryana, India', 'Farrukhnagar, Gurugram, Haryana, India']
You are an Image OSINT Investigator. I will provide you with an image and it's your job to determine where it was taken. By the end of your investigation you have to have at least a rough guess of the image's location. DECIDE ON ONE FINAL LOCATION, EVEN IF IT'S WRONG, JUST GUESS. Here's some AI-generated addresses that could be wrong to get you started. Choose one of these if you're uncertain:
1. Okaloosa County, Florida, United States
2. Gurugram, Haryana, India
3. Presidente Hayes, Región Occidental, Paraguay
4. Farrukhnagar, Gurugram, Haryana, India
5. Farrukhnagar, Gurugram, Haryana, India
 Based on the image provided, it appears to be a location with relatively flat terrain and sparse ve

  0%|          | 2/3000 [00:30<12:30:59, 15.03s/it]

...

 ```json
{
  "image_loc": "Turkey, [Unknown], Turkey"
}
```
['Niagara Region, Golden Horseshoe, Ontario, Canada', 'Niagara Region, Golden Horseshoe, Ontario, Canada', 'Niagara Region, Golden Horseshoe, Ontario, Canada', 'Niagara Region, Golden Horseshoe, Ontario, Canada', 'Niagara Region, Golden Horseshoe, Ontario, Canada']
You are an Image OSINT Investigator. I will provide you with an image and it's your job to determine where it was taken. By the end of your investigation you have to have at least a rough guess of the image's location. DECIDE ON ONE FINAL LOCATION, EVEN IF IT'S WRONG, JUST GUESS. Here's some AI-generated addresses that could be wrong to get you started. Choose one of these if you're uncertain:
1. Niagara Region, Golden Horseshoe, Ontario, Canada
2. Niagara Region, Golden Horseshoe, Ontario, Canada
3. Niagara Region, Golden Horseshoe, Ontario, Canada
4. Niagara Region, Golden Horseshoe, Ontario, Canada
5. Niagara Region, Golden Horseshoe, Ontario, Canada
 Based on th

  3%|▎         | 95/3000 [20:22<9:17:22, 11.51s/it] 

 {"image_loc": "Niagara Falls, Golden Horseshoe, Ontario, Canada"}
['Ba Dinh District, Hà Nội, Vietnam', 'Ba Dinh District, Hà Nội, Vietnam', 'Ba Dinh District, Hà Nội, Vietnam', 'Ba Dinh District, Hà Nội, Vietnam', 'Ba Dinh District, Hà Nội, Vietnam']
You are an Image OSINT Investigator. I will provide you with an image and it's your job to determine where it was taken. By the end of your investigation you have to have at least a rough guess of the image's location. DECIDE ON ONE FINAL LOCATION, EVEN IF IT'S WRONG, JUST GUESS. Here's some AI-generated addresses that could be wrong to get you started. Choose one of these if you're uncertain:
1. Ba Dinh District, Hà Nội, Vietnam
2. Ba Dinh District, Hà Nội, Vietnam
3. Ba Dinh District, Hà Nội, Vietnam
4. Ba Dinh District, Hà Nội, Vietnam
5. Ba Dinh District, Hà Nội, Vietnam
 Based on the image provided, which features a traditional Chinese building, likely a pagoda or temple, and what appears to be a scenic area with trees and waterways

  3%|▎         | 96/3000 [20:38<10:12:36, 12.66s/it]

 ```json
{
  "image_loc": "Hà Nội, Vietnam, China"
}
```
['Bronx County, The Bronx, New York, United States', 'Bronx County, The Bronx, New York, United States', 'Bronx County, The Bronx, New York, United States', 'Bronx County, The Bronx, New York, United States', 'Bronx County, The Bronx, New York, United States']
You are an Image OSINT Investigator. I will provide you with an image and it's your job to determine where it was taken. By the end of your investigation you have to have at least a rough guess of the image's location. DECIDE ON ONE FINAL LOCATION, EVEN IF IT'S WRONG, JUST GUESS. Here's some AI-generated addresses that could be wrong to get you started. Choose one of these if you're uncertain:
1. Bronx County, The Bronx, New York, United States
2. Bronx County, The Bronx, New York, United States
3. Bronx County, The Bronx, New York, United States
4. Bronx County, The Bronx, New York, United States
5. Bronx County, The Bronx, New York, United States


In [56]:
!rm /kaggle/working/imp2gps3k_output.csv